In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import nltk
import re
nltk.download('punkt')
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt_tab')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
!ls

archive.zip  sample_data


In [3]:
!unzip archive.zip

Archive:  archive.zip
  inflating: twitter_training.csv    
  inflating: twitter_validation.csv  


In [4]:
!ls

archive.zip  sample_data  twitter_training.csv	twitter_validation.csv


In [5]:
df_train= pd.read_csv('twitter_training.csv')
df_test=pd.read_csv('twitter_validation.csv')

In [6]:
df_train.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [7]:
df_test.head()

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [8]:
print(df_train.shape)
print(df_test.shape)

(74681, 4)
(999, 4)


In [9]:
df_train = df_train.sample(60000, random_state=42).reset_index(drop=True)   #We did this, b/c device had limited memory, and was taking really long for model implementation

In [10]:
df_train.shape

(60000, 4)

In [11]:
df_train = df_train.iloc[:, 2:]
df_test = df_test.iloc[:, 2:]

In [12]:
print(df_train.shape)
print(df_test.shape)

(60000, 2)
(999, 2)


In [13]:
df_train.columns = ["label", "text"]
df_test.columns = ["label", "text"]

In [14]:
df_train.head()

,label,text
0,Irrelevant,went to go in george's room to find his door w...
1,Positive,Yo this looks LIT! Team:GO/Overwatch combo
2,Negative,Pay attention executive administrators. While ...
3,Irrelevant,Guy looked at me and says my name was put on t...
4,Positive,one


In [15]:
df_test.head()

,label,text
0,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,Negative,@Microsoft Why do I pay for WORD when it funct...
2,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,Neutral,Now the President is slapping Americans in the...
4,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [16]:
df_train.isnull().sum()

,0
label,0
text,541


In [17]:
df_test.isnull().sum()

,0
label,0
text,0


In [18]:
df_train['text'] = df_train['text'].fillna('')    #Filling NaN with Empty String b/c nlp models can handle NaN values

In [19]:
df_train['text']=df_train['text'].str.lower()
df_test['text']=df_test['text'].str.lower()

In [20]:
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

In [21]:
df_train['text']=df_train['text'].apply(remove_html_tags)
df_test['text']=df_test['text'].apply(remove_html_tags)

In [22]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [23]:
df_train['text']=df_train['text'].apply(remove_url)
df_test['text']=df_test['text'].apply(remove_url)

In [24]:
df_train.head()

,label,text
0,Irrelevant,went to go in george's room to find his door w...
1,Positive,yo this looks lit! team:go/overwatch combo
2,Negative,pay attention executive administrators. while ...
3,Irrelevant,guy looked at me and says my name was put on t...
4,Positive,one


In [25]:
df_test.head()

,label,text
0,Neutral,bbc news - amazon boss jeff bezos rejects clai...
1,Negative,@microsoft why do i pay for word when it funct...
2,Negative,"csgo matchmaking is so full of closet hacking,..."
3,Neutral,now the president is slapping americans in the...
4,Negative,hi @eahelp i’ve had madeleine mccann in my cel...


In [26]:
exclude=string.punctuation
def remove_punc(text):
    return text.translate(str.maketrans('', '', exclude))

In [27]:
df_train['text']=df_train['text'].apply(remove_punc)
df_test['text']=df_test['text'].apply(remove_punc)

In [28]:
chat_words = {
    "A3": "Anytime, Anywhere, Anyplace",
    "ADIH": "Another Day In Hell",
    "AFK": "Away From Keyboard",
    "AFAIK": "As Far As I Know",
    "ASAP": "As Soon As Possible",
    "ASL": "Age, Sex, Location",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "BAE": "Before Anyone Else",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRUH": "Bro",
    "BRT": "Be Right There",
    "BSAAW": "Big Smile And A Wink",
    "BTW": "By The Way",
    "BWL": "Bursting With Laughter",
    "CSL": "Can’t Stop Laughing",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "DM": "Direct Message",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FIMH": "Forever In My Heart",
    "FOMO": "Fear Of Missing Out",
    "FR": "For Real",
    "FWIW": "For What It's Worth",
    "FYP": "For You Page",
    "FYI": "For Your Information",
    "G9": "Genius",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GMTA": "Great Minds Think Alike",
    "GN": "Good Night",
    "GOAT": "Greatest Of All Time",
    "GR8": "Great!",
    "HBD": "Happy Birthday",
    "IC": "I See",
    "ICQ": "I Seek You",
    "IDC": "I Don’t Care",
    "IDK": "I Don't Know",
    "IFYP": "I Feel Your Pain",
    "ILU": "I Love You",
    "ILY": "I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMU": "I Miss You",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "IYKYK": "If You Know, You Know",
    "JK": "Just Kidding",
    "KISS": "Keep It Simple, Stupid",
    "L": "Loss",
    "L8R": "Later",
    "LDR": "Long Distance Relationship",
    "LMK": "Let Me Know",
    "LMAO": "Laughing My A** Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "M8": "Mate",
    "MFW": "My Face When",
    "MID": "Mediocre",
    "MRW": "My Reaction When",
    "MTE": "My Thoughts Exactly",
    "NVM": "Never Mind",
    "NRN": "No Reply Necessary",
    "NPC": "Non-Player Character",
    "OIC": "Oh I See",
    "OP": "Overpowered",
    "PITA": "Pain In The A**",
    "POV": "Point Of View",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A** Off",
    "RN": "Right Now",
    "SK8": "Skate",
    "STATS": "Your Sex And Age",
    "SUS": "Suspicious",
    "TBH": "To Be Honest",
    "TFW": "That Feeling When",
    "THX": "Thank You",
    "TIME": "Tears In My Eyes",
    "TLDR": "Too Long, Didn’t Read",
    "TNTL": "Trying Not To Laugh",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "W": "Win",
    "W8": "Wait...",
    "WB": "Welcome Back",
    "WTF": "What The F**k",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "WYD": "What You Doing?",
    "WYWH": "Wish You Were Here",
    "ZZZ": "Sleeping, Bored, Tired"
}

In [29]:
def chat_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [30]:
df_train['text']=df_train['text'].apply(chat_conversion)
df_test['text']=df_test['text'].apply(chat_conversion)

In [31]:
df_train.head()

,label,text
0,Irrelevant,went to go in georges room to find his door wa...
1,Positive,yo this looks lit teamgooverwatch combo
2,Negative,pay attention executive administrators while y...
3,Irrelevant,guy looked at me and says my name was put on t...
4,Positive,one


In [32]:
df_test.head()

,label,text
0,Neutral,bbc news amazon boss jeff bezos rejects claims...
1,Negative,microsoft why do i pay for word when it functi...
2,Negative,csgo matchmaking is so full of closet hacking ...
3,Neutral,now the president is slapping americans in the...
4,Negative,hi eahelp i’ve had madeleine mccann in my cell...


In [33]:
stopwords.words('english')

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [34]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()   # split into words
    filtered = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered)

In [35]:
df_train['text']=df_train['text'].apply(remove_stopwords)
df_test['text']=df_test['text'].apply(remove_stopwords)

In [36]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 9.5 MB/s eta 0:00:00


In [37]:
#interpreting emojis
import emoji
df_train['text']=df_train['text'].apply(emoji.demojize)
df_test['text']=df_test['text'].apply(emoji.demojize)

In [38]:
df_train.head()

,label,text
0,Irrelevant,went go georges room find door locked climbed ...
1,Positive,yo looks lit teamgooverwatch combo
2,Negative,pay attention executive administrators stores ...
3,Irrelevant,guy looked says name put throw list lmaoooo ge...
4,Positive,one


In [39]:
df_test.head()

,label,text
0,Neutral,bbc news amazon boss jeff bezos rejects claims...
1,Negative,microsoft pay word functions poorly samsungus ...
2,Negative,csgo matchmaking full closet hacking truly awf...
3,Neutral,president slapping americans face really commi...
4,Negative,hi eahelp i’ve madeleine mccann cellar past 13...


In [40]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [41]:
#Without stemming
def process_overview_without_stemming(text):
    sentences = sent_tokenize(text)   # 1️⃣ sentence tokenize
    processed_sentences = []

    for sentence in sentences:
        words = word_tokenize(sentence)   # 2️⃣ word tokenize

        lemmatized = [lemmatizer.lemmatize(word) for word in words]  # 4️⃣ lemmatization

        processed_sentences.append(" ".join(lemmatized))

    return processed_sentences

In [42]:
df_train['text']=df_train['text'].apply(process_overview_without_stemming)
df_test['text']=df_test['text'].apply(process_overview_without_stemming)

In [43]:
df_train.head()

,label,text
0,Irrelevant,[went go george room find door locked climbed ...
1,Positive,[yo look lit teamgooverwatch combo]
2,Negative,[pay attention executive administrator store t...
3,Irrelevant,[guy looked say name put throw list lmaoooo ge...
4,Positive,[one]


In [44]:
df_test.head()

,label,text
0,Neutral,[bbc news amazon bos jeff bezos reject claim c...
1,Negative,[microsoft pay word function poorly samsungus ...
2,Negative,[csgo matchmaking full closet hacking truly aw...
3,Neutral,[president slapping american face really commi...
4,Negative,[hi eahelp i ’ ve madeleine mccann cellar past...


In [45]:
df_train['text'] = df_train['text'].apply(lambda x: " ".join(x))
df_test['text']  = df_test['text'].apply(lambda x: " ".join(x))

In [46]:
X_train = df_train['text']
y_train = df_train['label']
X_test = df_test['text']
y_test = df_test['label']

In [47]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=2,   #A term must appear in at least 2 documents in X_train.
    max_df=0.9  #The maximum number (or proportion) of documents a term can appear in to be included.
)

X_train = tfidf.fit_transform(X_train).toarray()
X_test  = tfidf.transform(X_test).toarray()

In [48]:
print(tfidf.idf_)   #IDF of each unique word
print(tfidf.get_feature_names_out())

[ 8.17347511 10.90350422 10.3926786  ... 10.90350422 10.90350422
 10.90350422]
['00' '00 000' '00 bst' ... 'นจาก' 'ℐℓ٥' 'ℐℓ٥ ٥υ']


In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression(max_iter=1000)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

  Irrelevant       0.98      0.93      0.95       171
    Negative       0.95      0.96      0.96       266
     Neutral       0.97      0.95      0.96       285
    Positive       0.94      0.97      0.96       277

    accuracy                           0.96       999
   macro avg       0.96      0.95      0.96       999
weighted avg       0.96      0.96      0.96       999



In [50]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

nb = MultinomialNB(alpha=0.1)   # alpha = smoothing

nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)

print(classification_report(y_test, y_pred_nb))

              precision    recall  f1-score   support

  Irrelevant       0.99      0.97      0.98       171
    Negative       0.98      0.98      0.98       266
     Neutral       0.99      0.97      0.98       285
    Positive       0.95      0.98      0.96       277

    accuracy                           0.97       999
   macro avg       0.98      0.97      0.98       999
weighted avg       0.98      0.97      0.98       999



Random Forests:

Do NOT scale well with 100k+ sparse features

Require dense input

Are usually slow and memory-heavy for text

In [51]:
from sklearn.feature_selection import SelectKBest, chi2

selector = SelectKBest(chi2, k=5000)

X_train_sel = selector.fit_transform(X_train, y_train)
X_test_sel  = selector.transform(X_test)

In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=30,
    n_jobs=-1,
    random_state=42
)

rf.fit(X_train_sel, y_train)
y_pred_rf = rf.predict(X_test_sel)

print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

  Irrelevant       1.00      0.12      0.21       171
    Negative       0.45      0.90      0.60       266
     Neutral       0.84      0.46      0.59       285
    Positive       0.69      0.71      0.70       277

    accuracy                           0.59       999
   macro avg       0.74      0.55      0.53       999
weighted avg       0.72      0.59      0.56       999

